# Surveying Telescopes

Analysing data from an optical interferometer to determine the 3-dimensional locations of the telescopes.

Data are a complete year (2012) of optical path delay observations from the CHARA interferometer in California (see http://www.chara.gsu.edu).